In [1]:
# import os
# import shutil
# # Define the directory containing the folders to delete
# root_dir = '../2'

# # List of folders to delete
# folders_to_delete = ['imgs', 'masks', 'params', 'results', 'testimgs']
# # folders_to_delete = ['1', '2', '3', '4', '5','6']

# # Loop through each folder and delete it if it is not empty
# for folder in folders_to_delete:
#     folder_path = os.path.join(root_dir, folder)
#     if os.path.isdir(folder_path) and os.listdir(folder_path):
#         # Delete the folder and its contents
#         shutil.rmtree(folder_path)

# # Print a message
# print(f'Deleted folders: {folders_to_delete}')

In [2]:
# import os
# import zipfile

# # Create a list of zip files and the target directory
# # zip_files = ['../1.zip', '../3.zip', '../4.zip', '../5.zip', '../6.zip']
# zip_files = ['../2.zip']
# target_dir = '../source/unzipFile'

# # Extract each zip file to the target directory
# for zip_file in zip_files:
#     with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#         zip_ref.extractall(target_dir)

# # Print a message
# print(f'Extracted all zip files to {target_dir}')

In [3]:
ls

drive/  sample_data/


In [4]:
cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [5]:
!pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 30.2 MB/s eta 0:00:00


In [6]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from data import *
from Net import *
from utils import *
import matplotlib.pyplot as plt

In [7]:
# 设置CUDA设备
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device}")

Using cuda


In [8]:
# filepath = r'../source/unzipFile'
# imagepath = r'../source/imgs'
# maskpath = r'../source/masks'
# sort_nii_image(filepath, imagepath, maskpath)

In [9]:
# 设置数据路径
dataPath = r'../source'
# 创建数据集
trainDataset = ImageDataSet(dataPath)
# 创建数据加载器
trainLoader = DataLoader(trainDataset, batch_size=2, shuffle=True)

In [10]:
# 获取trainDataset中的数据数量
dataset_size = len(trainDataset)
print(f'Train dataset size: {dataset_size}')

# 获取trainLoader中的批次数量
batch_size = len(trainLoader)
print(f'Train loader batch size: {batch_size}')

Train dataset size: 58
Train loader batch size: 29


In [11]:
for i, data in enumerate(trainLoader):
  inputs, labels = data
  print(inputs.shape)
  print(labels.shape)
  break

torch.Size([2, 1, 256, 224, 176])
torch.Size([2, 1, 256, 224, 176])


In [12]:
# input_shapes = []
# label_shapes = []

# for i, data in enumerate(trainLoader):
#   inputs, labels = data
#   input_shapes.append(inputs.shape)
#   label_shapes.append(labels.shape)

# # Check if all inputs have the same shape
# if len(set(input_shapes)) != 1:
#   print("Inconsistent input shapes")

# # Check if all labels have the same shape
# if len(set(label_shapes)) != 1:
#   print("Inconsistent label shapes")

# # Print the unique shapes
# print("Input shapes:", set(input_shapes))
# print("Label shapes:", set(label_shapes))

In [13]:
import pdb

In [14]:
# 创建网络模型
net = UNet().to(device)
#net = net.half()  # 将网络转换为半精度

# 定义损失函数和优化器
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.MSELoss()
# criterion =nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 用于保存最低损失
best_loss = float('inf')


In [15]:
# !pip install tensorboardX
# !pip install tensorboard
# !pip install tensorflow

In [16]:
# from torch.utils.tensorboard import SummaryWriter

# # default `log_dir` is "runs" - we'll be more specific here
# writer = SummaryWriter('runs/fk')
# # Load the TensorBoard notebook extension
# %load_ext tensorboard
# %tensorboard --logdir=runs

In [20]:
#pdb.set_trace()
accumulation_steps = 4  # 假设我们想要每累积4个小批量执行一次权重更新
# 训练网络
for epoch in range(10):  # 根据您的需求设置训练的轮数
    epoch_loss=0
    optimizer.zero_grad()  # 在开始梯度累积之前，先清零梯度
    for i, data in enumerate(trainLoader):
        inputs, labels = data
        #inputs, labels = inputs.half(), labels.half()  # 将输入和标签转换为半精度
        inputs, labels = inputs.to(device), labels.to(device)
        # 将labels中所有非零的元素都设置为1
        # labels = torch.where(labels != 0, torch.tensor(1, device=device), labels)
        #print(labels)

        # 前向传播
        outputs = net(inputs)

        # labels = labels.permute(0,1, 4, 2, 3)
        # 计算损失
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()

        # 反向传播和优化
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # # 判断是否执行优化步骤
        # if (i + 1) % accumulation_steps == 0 or (i + 1) == len(trainLoader):
        #     optimizer.step()
        #     optimizer.zero_grad()


        print(f'Epoch {epoch+1}/{10}, Step {i+1}/{len(trainLoader)}, Loss: {loss.item()}')

    # 计算平均损失
    avg_loss = epoch_loss / len(trainLoader)
    # ...log the running loss
    # writer.add_scalar('training loss',avg_loss, epoch)
    # 如果平均损失更低，保存模型
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(net.state_dict(), f'../source/params/best_model_epoch_{epoch+1}.pth')

print('Finished Training')

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.20 GiB. GPU 0 has a total capacity of 39.56 GiB of which 344.81 MiB is free. Process 72219 has 39.22 GiB memory in use. Of the allocated memory 38.11 GiB is allocated by PyTorch, and 622.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)